<a href="https://colab.research.google.com/github/JdoubleU2/CareConnect/blob/main/finetune/src/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
# Normally using pip install unsloth is enough

# Temporarily as of Jan 31st 2025, Colab has some issues with Pytorch
# Using pip install unsloth will take 3 minutes, whilst the below takes <1 minute:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

Add LoRA adapters so we only need to update 1 to 10% of all parameters! This techniques allow us to fine-tune large models efficiently by adjusting only a subset of parameters instead of the entire model, reducing computational cost and memory usage.

In [3]:

# Use PEFT (Parameter-Efficient Fine-Tunning) on our selected model
model = FastLanguageModel.get_peft_model(
    model,
    # rank of the LoRA matrix. The rank determines the number of parameters added for adaptation.
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128

    # which parts (modules) of the model to apply LoRA fine-tuning.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],

    lora_alpha = 16, #how much influence the fine-tuned weights have on the original model.

    #Apply dropout to LoRA layers during fine-tuning to prevent overfitting
    lora_dropout = 0, # Supports any, but = 0 is optimized

    bias = "none",    # Supports any, but = "none" is optimized

    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context

    random_state = 3407, #seed for reproducibility, ensuring that results remain consistent across runs.

    # Rank-Stabilized LoRA (RS-LoRA), which helps stabilize training when fine-tuning models with very high rank
    use_rslora = False,  #  disable support rank stabilized LoRA (default)

    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


DATA PROCESSING Section

In [4]:
import json
import os
from datasets import Dataset

dataset = []
directory_path = "/content/Training_Data/"

def convert_to_conversation(sample):
    conversation = [
        {
            "from": "system",
            "value": sample['instruction']
        },
        {
            "from": "human",
            "value": "Predict the disease based on these symptoms: " + sample['input']
        },
        {
            "from": "gpt",
            "value": sample['output']
        }
    ]
    return {"conversations": conversation}

for file in os.listdir(directory_path):
    if file.endswith(".jsonl"): #find all datasets
        file_path = os.path.join(directory_path, file)
        with open(file_path, "r", encoding="utf-8") as file:
            for line in file:
                dataset.append(json.loads(line.strip()))    #Append dataset to training dataset


data_temp = [convert_to_conversation(data) for data in dataset] #Coverting dataset to conversation format
data_dict = {
    "conversations": [item["conversations"] for item in data_temp]
}

# Added missing closing parenthesis
dataset = Dataset.from_dict(data_dict)

In [5]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass


In [6]:
print(dataset[:5])


{'conversations': [[{'from': 'system', 'value': 'Given the name of a medication, provide a brief description of its primary use or therapeutic purpose.'}, {'from': 'human', 'value': 'Predict the disease based on these symptoms: Prohibin 20mg Capsule'}, {'from': 'gpt', 'value': 'Treatment of HeartburnTreatment of Gastroesophageal reflux disease (Acid reflux)Treatment of Peptic ulcer diseaseTreatment of Zollinger-Ellison syndrome'}], [{'from': 'system', 'value': 'Given the name of a medication, provide a brief description of its primary use or therapeutic purpose.'}, {'from': 'human', 'value': 'Predict the disease based on these symptoms: Paracain Ophthalmic Solution'}, {'from': 'gpt', 'value': 'Eye anesthesia'}], [{'from': 'system', 'value': 'Given the name of a medication, provide a brief description of its primary use or therapeutic purpose.'}, {'from': 'human', 'value': 'Predict the disease based on these symptoms: Biospor 1% Cream'}, {'from': 'gpt', 'value': 'Fungal skin infections'

In [7]:
from unsloth.chat_templates import standardize_sharegpt

dataset = standardize_sharegpt(dataset) #Ensures the dataset is in a clean and consistent format
dataset = dataset.map(formatting_prompts_func, batched = True,) #Converts the standardized data into the final format expected by the LLaMA 3.2 chat template.

Standardizing format:   0%|          | 0/771379 [00:00<?, ? examples/s]

Map:   0%|          | 0/771379 [00:00<?, ? examples/s]

In [8]:
dataset[5] ["conversations"] #  raw data for debugging or validation.

[{'content': 'Given the name of a medication, provide a brief description of its primary use or therapeutic purpose.',
  'role': 'system'},
 {'content': 'Predict the disease based on these symptoms: Amrox Junior Syrup',
  'role': 'user'},
 {'content': 'Treatment of Cough with mucus', 'role': 'assistant'}]

In [9]:
dataset[5]["text"] # verify that the formatting and preprocessing worked as expected.

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nGiven the name of a medication, provide a brief description of its primary use or therapeutic purpose.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nPredict the disease based on these symptoms: Amrox Junior Syrup<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nTreatment of Cough with mucus<|eot_id|>'

Training setup


In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,  #model to finetune  (were using llama3.2)
    tokenizer = tokenizer, #selects our toenizer from the code block above
    train_dataset = dataset, #basially all this stuff in predefined in the blocks above just ctl+click
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 1,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(# alright now it gets freaky
        per_device_train_batch_size = 2,    #each GPU processes two examples per bactch
        gradient_accumulation_steps = 4,    #  Simulates a batch size of 2 × 4 = 8 by accumulating gradients over 4 steps before updating weights.
        warmup_steps = 5,   #this will gradually increse the learning rate after the first 5 steps
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 100, #attempt at 100 steps of training. Will modify this later
        learning_rate = 2e-4, # initial learning rate.
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,  #logs training metrics
        optim = "adamw_8bit",   #this is a optimizer with 8-bit weights. this is reduced for memory usage
        weight_decay = 0.01,    #this helps overfitting, applies a weight decay to regualize the model
        lr_scheduler_type = "linear",
        seed = 3407,    #Sets the random seed for reproducibility
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Converting train dataset to ChatML:   0%|          | 0/771379 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/771379 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/771379 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/771379 [00:00<?, ? examples/s]

In [13]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/771379 [00:00<?, ? examples/s]

Very masking is done correctly

In [14]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nGiven the name of a medication, provide a brief description of its primary use or therapeutic purpose.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nPredict the disease based on these symptoms: Amrox Junior Syrup<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nTreatment of Cough with mucus<|eot_id|>'

In [15]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                   \n\nTreatment of Cough with mucus<|eot_id|>'

In [16]:
#Show Computer memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
2.301 GB of memory reserved.


In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 771,379 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,5.716900
2,3.886200
3,3.608500
4,2.221300
5,3.092000
6,3.113200
7,3.146600
8,2.042100
9,1.952500
10,1.432600


More memory stats

In [18]:
import torch

used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

325.0105 seconds used for training.
5.42 minutes used for training.
Peak reserved memory = 4.012 GB.
Peak reserved memory for training = 1.711 GB.
Peak reserved memory % of max memory = 27.217 %.
Peak reserved memory for training % of max memory = 11.607 %.


# New Section

Local Saving

In [20]:
from huggingface_hub import login
#put your login stuff here
from google.colab import userdata
token= userdata.get('huggingface')
login(token)
model.save_pretrained_gguf("careconnect-llama3.2-3b",tokenizer, push_to_hub = True)


Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.4G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 0.85 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:01<00:00, 20.17it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving careconnect-llama3.2-3b/pytorch_model-00001-of-00002.bin...
Unsloth: Saving careconnect-llama3.2-3b/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


...

unsloth.Q8_0.gguf:   0%|          | 0.00/3.42G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/JdoubleU/careconnect-llama3.2-3b


No files have been modified since last commit. Skipping to prevent empty commit.


Saved Ollama Modelfile to https://huggingface.co/JdoubleU/careconnect-llama3.2-3b


In [21]:
import os
# Upload to Hugging Face
model.push_to_hub("careconnect-llama3.2-3b",token = os.getenv("HUGGINGFACE_TOKEN"))
tokenizer.push_to_hub("careconnect-llama3.2-3b",token = os.getenv("HUGGINGFACE_TOKEN"))
model.push_to_hub_gguf(
    "careconnect-llama3.2-3b",
    tokenizer,
    quantization_method = ["q4_k_m", "q8_0", "q5_k_m"],
    token = os.getenv("HUGGINGFACE_TOKEN")
    )

README.md:   0%|          | 0.00/614 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

Saved model to https://huggingface.co/careconnect-llama3.2-3b


...

unsloth.Q5_K_M.gguf:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/JdoubleU/careconnect-llama3.2-3b


No files have been modified since last commit. Skipping to prevent empty commit.


Saved Ollama Modelfile to https://huggingface.co/JdoubleU/careconnect-llama3.2-3b
